Script de scrapping de BigCityLife

# Import librairies

In [ ]:
import pandas as pd
from utils.scrapping_functions import *
from utils.cleaning_functions import *

# Get event list and link

In [1]:
url = 'https://www.bigcitylife.fr/agenda/'

In [4]:
soup = fetch_soup(url)

In [5]:
event_tag = soup.find('div', class_="tribe-events-calendar-list")
event_list = event_tag.find_all('div', class_='tribe-common-g-row tribe-events-calendar-list__event-row')
event_link = []
event_name_list = []
for event in event_list :
    link = event.find('h3', class_='tribe-events-calendar-list__event-title tribe-common-h6 tribe-common-h4--min-medium').find('a')['href']
    event_link.append(link)
    name = event.find('h3', class_='tribe-events-calendar-list__event-title tribe-common-h6 tribe-common-h4--min-medium').text
    event_name_list.append(name)

In [6]:
df = pd.DataFrame({'events' : event_name_list,
                   'event_link': event_link})

In [7]:
df.head()

,events,event_link
0,Exposition Chevaliers,https://www.bigcitynantes.fr/agenda/nouvelle-e...
1,Spectacle son et lumière Immersia,https://www.bigcitynantes.fr/agenda/premiere-d...
2,La Quinzaine féministe et des égalités au Th...,https://www.bigcitynantes.fr/agenda/la-quinzai...
3,Festival Eurofonik #13 : Musiques des mondes...,https://www.bigcitynantes.fr/agenda/festival-e...
4,Immersion au cœur du Ministère de l’Europe e...,https://www.bigcitynantes.fr/agenda/immersion-...


# Get info event

In [8]:
df['soup_event'] = df['event_link'].apply(fetch_soup)

In [9]:
df.head()

,events,event_link,soup_event
0,Exposition Chevaliers,https://www.bigcitynantes.fr/agenda/nouvelle-e...,"[html, [[<meta charset=""utf-8""/>, <meta conten..."
1,Spectacle son et lumière Immersia,https://www.bigcitynantes.fr/agenda/premiere-d...,"[html, [[<meta charset=""utf-8""/>, <meta conten..."
2,La Quinzaine féministe et des égalités au Th...,https://www.bigcitynantes.fr/agenda/la-quinzai...,"[html, [[<meta charset=""utf-8""/>, <meta conten..."
3,Festival Eurofonik #13 : Musiques des mondes...,https://www.bigcitynantes.fr/agenda/festival-e...,"[html, [[<meta charset=""utf-8""/>, <meta conten..."
4,Immersion au cœur du Ministère de l’Europe e...,https://www.bigcitynantes.fr/agenda/immersion-...,"[html, [[<meta charset=""utf-8""/>, <meta conten..."


In [10]:
df['event_id'] = df['soup_event'].apply(fetch_event_id)

In [11]:
df['place'] = df['soup_event'].apply(fetch_place)

In [12]:
df['coordinates'] = df['soup_event'].apply(fetch_coordinate)

In [13]:
df['tags'] = df['soup_event'].apply(fetch_event_tag)

In [14]:
df['price'] = df['soup_event'].apply(fetch_price)

In [15]:
df['description'] = df.apply(lambda row: fetch_description(row['soup_event'], row['event_id']), axis=1)

In [16]:
df['image'] = df.apply(lambda row: fetch_image(row['soup_event'], row['event_id']), axis=1)

In [17]:
df['adress'] = df['coordinates'].apply(fetch_adress)

In [18]:
df['date'] = df['soup_event'].apply(fetch_date)

In [19]:
df[['date_debut', 'date_fin']] = df['date'].apply(extraire_dates)

In [20]:
df['date_debut'] = pd.to_datetime(df['date_debut'], errors='coerce')
df['date_fin'] = pd.to_datetime(df['date_fin'], errors='coerce')

In [21]:
df.head()

,events,event_link,soup_event,event_id,place,coordinates,tags,price,description,image,adress,date,date_debut,date_fin
0,Exposition Chevaliers,https://www.bigcitynantes.fr/agenda/nouvelle-e...,"[html, [[<meta charset=""utf-8""/>, <meta conten...",106373,Château des Ducs de Bretagne,"[-1.5490891, 47.2155761]",[EXPO],PAYANT,Chevaliers est une exposition constituée d’un ...,https://www.bigcitynantes.fr/wp-content/upload...,"2 Rue des Etats, 44000, Nantes",DU 19 OCTOBRE AU 20 AVR.,2025-10-19,2025-04-20
1,Spectacle son et lumière Immersia,https://www.bigcitynantes.fr/agenda/premiere-d...,"[html, [[<meta charset=""utf-8""/>, <meta conten...",106754,Eglise Notre-Dame-de-Bon-Port,"[-1.5689119, 47.209975]",[SPECTACLE],PAYANT,"L’écrin du dôme de Notre Dame-de-Bon-Port, va ...",https://www.bigcitynantes.fr/wp-content/upload...,"5 Place du Sanitat, 44100, Nantes",DU 14 NOVEMBRE AU 29 MAR.,2025-11-14,2025-03-29
2,La Quinzaine féministe et des égalités au Th...,https://www.bigcitynantes.fr/agenda/la-quinzai...,"[html, [[<meta charset=""utf-8""/>, <meta conten...",110451,Théâtre 100 Noms,"[-1.5736452, 47.2006731]","[CONCERT, CULTURE, HUMOUR, IMPRO, SPECTACLE, T...",PAYANT,Le combat pour l’égalité Femme-Homme se poursu...,https://www.bigcitynantes.fr/wp-content/upload...,"21 Quai des Antilles, 44200, Nantes",DU 2 MARS AU 15 MAR.,2025-03-02,2025-03-15
3,Festival Eurofonik #13 : Musiques des mondes...,https://www.bigcitynantes.fr/agenda/festival-e...,"[html, [[<meta charset=""utf-8""/>, <meta conten...",109504,Le nouveau Pavillon,"[-1.5885849, 47.1813796]","[CONCERT, CULTURE, DANSE, EUROPE, MUSIQUE, REN...",PAYANT,La 13e édition d’Eurofonik débarque du 5 au 16...,https://www.bigcitynantes.fr/wp-content/upload...,"2 Rue Célestin Freinet, 44340, Bouguenais",DU 5 MARS AU 16 MAR.,2025-03-05,2025-03-16
4,Immersion au cœur du Ministère de l’Europe e...,https://www.bigcitynantes.fr/agenda/immersion-...,"[html, [[<meta charset=""utf-8""/>, <meta conten...",110694,Espace Cosmopolis,"[-1.5620911, 47.2140864]","[CONFERENCE, CULTURE, DECOUVERTE, ECHANGE, ESC...",GRATUIT,"Du 11 au 15 mars 2025, le coeur de la démocrat...",https://www.bigcitynantes.fr/wp-content/upload...,"16 Rue Scribe, 44000, Nantes",DU 11 MARS AU 15 MAR.,2025-03-11,2025-03-15


# Cleaning

In [22]:
df.rename(columns={
    'events' : 'Nom',
    'place' : 'Lieu',
    'tags' :'Tags',
    'price' : 'Tarif',
    'adress' : 'Adresse',
    'date_debut' : 'DateDebut',
    'date_fin' : 'DateFin',
    'description' : 'Description',
    'image' : 'Image'
}, inplace=True)

In [23]:
df['id'] = ''
df['IdCreateur'] = ''
df['CreatedAt'] = datetime.now()
df['UpdatedAt'] = datetime.now()

In [24]:
df.head()

,Nom,event_link,soup_event,event_id,Lieu,coordinates,Tags,Tarif,Description,Image,Adresse,date,DateDebut,DateFin,id,IdCreateur,CreatedAt,UpdatedAt
0,Exposition Chevaliers,https://www.bigcitynantes.fr/agenda/nouvelle-e...,"[html, [[<meta charset=""utf-8""/>, <meta conten...",106373,Château des Ducs de Bretagne,"[-1.5490891, 47.2155761]",[EXPO],PAYANT,Chevaliers est une exposition constituée d’un ...,https://www.bigcitynantes.fr/wp-content/upload...,"2 Rue des Etats, 44000, Nantes",DU 19 OCTOBRE AU 20 AVR.,2025-10-19,2025-04-20,,,2025-03-13 12:23:51.319205,2025-03-13 12:23:51.319823
1,Spectacle son et lumière Immersia,https://www.bigcitynantes.fr/agenda/premiere-d...,"[html, [[<meta charset=""utf-8""/>, <meta conten...",106754,Eglise Notre-Dame-de-Bon-Port,"[-1.5689119, 47.209975]",[SPECTACLE],PAYANT,"L’écrin du dôme de Notre Dame-de-Bon-Port, va ...",https://www.bigcitynantes.fr/wp-content/upload...,"5 Place du Sanitat, 44100, Nantes",DU 14 NOVEMBRE AU 29 MAR.,2025-11-14,2025-03-29,,,2025-03-13 12:23:51.319205,2025-03-13 12:23:51.319823
2,La Quinzaine féministe et des égalités au Th...,https://www.bigcitynantes.fr/agenda/la-quinzai...,"[html, [[<meta charset=""utf-8""/>, <meta conten...",110451,Théâtre 100 Noms,"[-1.5736452, 47.2006731]","[CONCERT, CULTURE, HUMOUR, IMPRO, SPECTACLE, T...",PAYANT,Le combat pour l’égalité Femme-Homme se poursu...,https://www.bigcitynantes.fr/wp-content/upload...,"21 Quai des Antilles, 44200, Nantes",DU 2 MARS AU 15 MAR.,2025-03-02,2025-03-15,,,2025-03-13 12:23:51.319205,2025-03-13 12:23:51.319823
3,Festival Eurofonik #13 : Musiques des mondes...,https://www.bigcitynantes.fr/agenda/festival-e...,"[html, [[<meta charset=""utf-8""/>, <meta conten...",109504,Le nouveau Pavillon,"[-1.5885849, 47.1813796]","[CONCERT, CULTURE, DANSE, EUROPE, MUSIQUE, REN...",PAYANT,La 13e édition d’Eurofonik débarque du 5 au 16...,https://www.bigcitynantes.fr/wp-content/upload...,"2 Rue Célestin Freinet, 44340, Bouguenais",DU 5 MARS AU 16 MAR.,2025-03-05,2025-03-16,,,2025-03-13 12:23:51.319205,2025-03-13 12:23:51.319823
4,Immersion au cœur du Ministère de l’Europe e...,https://www.bigcitynantes.fr/agenda/immersion-...,"[html, [[<meta charset=""utf-8""/>, <meta conten...",110694,Espace Cosmopolis,"[-1.5620911, 47.2140864]","[CONFERENCE, CULTURE, DECOUVERTE, ECHANGE, ESC...",GRATUIT,"Du 11 au 15 mars 2025, le coeur de la démocrat...",https://www.bigcitynantes.fr/wp-content/upload...,"16 Rue Scribe, 44000, Nantes",DU 11 MARS AU 15 MAR.,2025-03-11,2025-03-15,,,2025-03-13 12:23:51.319205,2025-03-13 12:23:51.319823


In [25]:
df['Heure de début'] = df.apply(lambda x : '10:00' if 'EXPO' in x['Tags'] else '19:00',axis=1)

In [ ]:
# Nettoyage des colonnes
df['DateDebut'] = df['DateDebut'].astype(str).str.strip()
df['DateFin'] = df['DateFin'].astype(str).str.strip()
df['Heure de début'] = df['Heure de début'].astype(str).str.strip()

# Création du datetime pour DateDebut pour coller avec les attentes de la base de données
df['DateDebut'] = pd.to_datetime(
    df['DateDebut'] + ' ' + df['Heure de début'], 
    format='%Y-%m-%d %H:%M',
    errors='coerce'
)

# Création du datetime pour DateFin pour coller avec les attentes de la base de données
df['DateFin'] = pd.to_datetime(
    df['DateFin'] + ' 23:59:59',
    format='%Y-%m-%d %H:%M:%S',
    errors='coerce'
)

In [27]:
df.head()

,Nom,event_link,soup_event,event_id,Lieu,coordinates,Tags,Tarif,Description,Image,Adresse,date,DateDebut,DateFin,id,IdCreateur,CreatedAt,UpdatedAt,Heure de début
0,Exposition Chevaliers,https://www.bigcitynantes.fr/agenda/nouvelle-e...,"[html, [[<meta charset=""utf-8""/>, <meta conten...",106373,Château des Ducs de Bretagne,"[-1.5490891, 47.2155761]",[EXPO],PAYANT,Chevaliers est une exposition constituée d’un ...,https://www.bigcitynantes.fr/wp-content/upload...,"2 Rue des Etats, 44000, Nantes",DU 19 OCTOBRE AU 20 AVR.,2025-10-19 10:00:00,2025-04-20 23:59:59,,,2025-03-13 12:23:51.319205,2025-03-13 12:23:51.319823,10:00
1,Spectacle son et lumière Immersia,https://www.bigcitynantes.fr/agenda/premiere-d...,"[html, [[<meta charset=""utf-8""/>, <meta conten...",106754,Eglise Notre-Dame-de-Bon-Port,"[-1.5689119, 47.209975]",[SPECTACLE],PAYANT,"L’écrin du dôme de Notre Dame-de-Bon-Port, va ...",https://www.bigcitynantes.fr/wp-content/upload...,"5 Place du Sanitat, 44100, Nantes",DU 14 NOVEMBRE AU 29 MAR.,2025-11-14 19:00:00,2025-03-29 23:59:59,,,2025-03-13 12:23:51.319205,2025-03-13 12:23:51.319823,19:00
2,La Quinzaine féministe et des égalités au Th...,https://www.bigcitynantes.fr/agenda/la-quinzai...,"[html, [[<meta charset=""utf-8""/>, <meta conten...",110451,Théâtre 100 Noms,"[-1.5736452, 47.2006731]","[CONCERT, CULTURE, HUMOUR, IMPRO, SPECTACLE, T...",PAYANT,Le combat pour l’égalité Femme-Homme se poursu...,https://www.bigcitynantes.fr/wp-content/upload...,"21 Quai des Antilles, 44200, Nantes",DU 2 MARS AU 15 MAR.,2025-03-02 19:00:00,2025-03-15 23:59:59,,,2025-03-13 12:23:51.319205,2025-03-13 12:23:51.319823,19:00
3,Festival Eurofonik #13 : Musiques des mondes...,https://www.bigcitynantes.fr/agenda/festival-e...,"[html, [[<meta charset=""utf-8""/>, <meta conten...",109504,Le nouveau Pavillon,"[-1.5885849, 47.1813796]","[CONCERT, CULTURE, DANSE, EUROPE, MUSIQUE, REN...",PAYANT,La 13e édition d’Eurofonik débarque du 5 au 16...,https://www.bigcitynantes.fr/wp-content/upload...,"2 Rue Célestin Freinet, 44340, Bouguenais",DU 5 MARS AU 16 MAR.,2025-03-05 19:00:00,2025-03-16 23:59:59,,,2025-03-13 12:23:51.319205,2025-03-13 12:23:51.319823,19:00
4,Immersion au cœur du Ministère de l’Europe e...,https://www.bigcitynantes.fr/agenda/immersion-...,"[html, [[<meta charset=""utf-8""/>, <meta conten...",110694,Espace Cosmopolis,"[-1.5620911, 47.2140864]","[CONFERENCE, CULTURE, DECOUVERTE, ECHANGE, ESC...",GRATUIT,"Du 11 au 15 mars 2025, le coeur de la démocrat...",https://www.bigcitynantes.fr/wp-content/upload...,"16 Rue Scribe, 44000, Nantes",DU 11 MARS AU 15 MAR.,2025-03-11 10:00:00,2025-03-15 23:59:59,,,2025-03-13 12:23:51.319205,2025-03-13 12:23:51.319823,10:00


In [28]:
df.drop(columns=['event_link','soup_event', 'coordinates', 'coordinates','date', 'event_id','Heure de début'], inplace = True)

In [29]:
df.head()

,Nom,Lieu,Tags,Tarif,Description,Image,Adresse,DateDebut,DateFin,id,IdCreateur,CreatedAt,UpdatedAt
0,Exposition Chevaliers,Château des Ducs de Bretagne,[EXPO],PAYANT,Chevaliers est une exposition constituée d’un ...,https://www.bigcitynantes.fr/wp-content/upload...,"2 Rue des Etats, 44000, Nantes",2025-10-19 10:00:00,2025-04-20 23:59:59,,,2025-03-13 12:23:51.319205,2025-03-13 12:23:51.319823
1,Spectacle son et lumière Immersia,Eglise Notre-Dame-de-Bon-Port,[SPECTACLE],PAYANT,"L’écrin du dôme de Notre Dame-de-Bon-Port, va ...",https://www.bigcitynantes.fr/wp-content/upload...,"5 Place du Sanitat, 44100, Nantes",2025-11-14 19:00:00,2025-03-29 23:59:59,,,2025-03-13 12:23:51.319205,2025-03-13 12:23:51.319823
2,La Quinzaine féministe et des égalités au Th...,Théâtre 100 Noms,"[CONCERT, CULTURE, HUMOUR, IMPRO, SPECTACLE, T...",PAYANT,Le combat pour l’égalité Femme-Homme se poursu...,https://www.bigcitynantes.fr/wp-content/upload...,"21 Quai des Antilles, 44200, Nantes",2025-03-02 19:00:00,2025-03-15 23:59:59,,,2025-03-13 12:23:51.319205,2025-03-13 12:23:51.319823
3,Festival Eurofonik #13 : Musiques des mondes...,Le nouveau Pavillon,"[CONCERT, CULTURE, DANSE, EUROPE, MUSIQUE, REN...",PAYANT,La 13e édition d’Eurofonik débarque du 5 au 16...,https://www.bigcitynantes.fr/wp-content/upload...,"2 Rue Célestin Freinet, 44340, Bouguenais",2025-03-05 19:00:00,2025-03-16 23:59:59,,,2025-03-13 12:23:51.319205,2025-03-13 12:23:51.319823
4,Immersion au cœur du Ministère de l’Europe e...,Espace Cosmopolis,"[CONFERENCE, CULTURE, DECOUVERTE, ECHANGE, ESC...",GRATUIT,"Du 11 au 15 mars 2025, le coeur de la démocrat...",https://www.bigcitynantes.fr/wp-content/upload...,"16 Rue Scribe, 44000, Nantes",2025-03-11 10:00:00,2025-03-15 23:59:59,,,2025-03-13 12:23:51.319205,2025-03-13 12:23:51.319823


In [30]:
df.columns

Index(['Nom', 'Lieu', 'Tags', 'Tarif', 'Description', 'Image', 'Adresse',
       'DateDebut', 'DateFin', 'id', 'IdCreateur', 'CreatedAt', 'UpdatedAt'],
      dtype='object')

In [31]:
df.to_csv('/Users/manu/Desktop/SUP/Semaine Campus/OUTPUT/output2.csv',index=False)